Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [6]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [0]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)


In [0]:
train = df[pd.DatetimeIndex(df['SALE_DATE']).month.isin([1,2,3])] # Train the model with Jan, Feb, March data
test = df[pd.DatetimeIndex(df['SALE_DATE']).month.isin([4])] # Test with April

In [12]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,18167.0,4.447262e+03,3.679406e+03,1.0,1343.0,3569.0,6656.0,16350.0
LOT,18167.0,3.436415e+02,6.061895e+02,1.0,21.0,49.0,286.0,9022.0
EASE-MENT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,18167.0,1.078270e+04,1.121115e+03,0.0,10306.0,11210.0,11360.0,11697.0
RESIDENTIAL_UNITS,18167.0,1.721418e+00,9.381721e+00,0.0,0.0,1.0,2.0,750.0
COMMERCIAL_UNITS,18167.0,2.989486e-01,6.087744e+00,-1.0,0.0,0.0,0.0,570.0
TOTAL_UNITS,18167.0,2.172235e+00,1.166344e+01,0.0,1.0,1.0,2.0,755.0
GROSS_SQUARE_FEET,18167.0,3.214913e+03,2.155829e+04,0.0,528.0,1368.0,2273.5,1303935.0
YEAR_BUILT,18162.0,1.822193e+03,4.836412e+02,0.0,1920.0,1940.0,1965.0,2019.0
TAX_CLASS_AT_TIME_OF_SALE,18167.0,1.617053e+00,8.073494e-01,1.0,1.0,1.0,2.0,4.0


In [18]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
BOROUGH,18167,5,4,5883,NaT,NaT
TAX_CLASS_AT_PRESENT,18167,10,1,8911,NaT,NaT
NEIGHBORHOOD,18167,11,OTHER,15058,NaT,NaT
BUILDING_CLASS_CATEGORY,18167,43,01 ONE FAMILY DWELLINGS,4094,NaT,NaT
SALE_DATE,18167,90,2019-01-24 00:00:00,480,2019-01-01,2019-03-31
BUILDING_CLASS_AT_PRESENT,18167,140,D4,2640,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,18167,140,D4,2640,NaT,NaT
APARTMENT_NUMBER,3972,1450,4,81,NaT,NaT
LAND_SQUARE_FEET,18139,3207,0,5784,NaT,NaT
ADDRESS,18167,17926,N/A ROCKAWAY BOULEVARD,5,NaT,NaT


In [15]:
train.groupby('BOROUGH')['SALE_PRICE'].mean()

BOROUGH
1    3.200017e+06
2    5.736442e+05
3    8.495242e+05
4    7.335611e+05
5    3.947132e+05
Name: SALE_PRICE, dtype: float64

In [0]:
target = 'SALE_PRICE'
high_cardinality = ['TAX_CLASS_AT_PRESENT', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 'SALE_DATE', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'APARTMENT_NUMBER', 'LAND_SQUARE_FEET', 'ADDRESS', 'EASE-MENT']
#high_cardinality = ['BUILDING_CLASS_CATEGORY', 'SALE_DATE', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'APARTMENT_NUMBER', 'LAND_SQUARE_FEET', 'ADDRESS']
features = train.columns.drop([target] + high_cardinality)

In [0]:
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [75]:
X_train.head(10)

,BOROUGH_1,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
0,1,0,0,0,0,716,1246,10011.0,1.0,0.0,1.0,1979.0,2007.0,2
1,1,0,0,0,0,812,68,10018.0,0.0,6.0,6.0,15435.0,1920.0,4
2,1,0,0,0,0,839,69,10018.0,0.0,7.0,7.0,11332.0,1930.0,4
3,1,0,0,0,0,592,1041,10014.0,1.0,0.0,1.0,500.0,0.0,2
4,1,0,0,0,0,1379,1402,10065.0,1.0,0.0,1.0,6406.0,0.0,2
5,1,0,0,0,0,1551,131,10128.0,10.0,0.0,10.0,6570.0,1920.0,2
6,1,0,0,0,0,1891,159,10025.0,10.0,0.0,10.0,5810.0,1900.0,2
7,0,1,0,0,0,4090,37,10461.0,1.0,0.0,1.0,1328.0,1925.0,1
8,0,1,0,0,0,4120,18,10461.0,1.0,0.0,1.0,1728.0,1935.0,1
9,0,1,0,0,0,4120,20,10461.0,1.0,0.0,1.0,1728.0,1935.0,1


In [0]:
X_test.head(10)

In [0]:
import category_encoders as cat_enc

encoder = cat_enc.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test =  encoder.transform(X_test)


In [84]:
X_train.dtypes

BOROUGH_1                      int64
BOROUGH_2                      int64
BOROUGH_3                      int64
BOROUGH_4                      int64
BOROUGH_5                      int64
BLOCK                          int64
LOT                            int64
ZIP_CODE                     float64
RESIDENTIAL_UNITS            float64
COMMERCIAL_UNITS             float64
TOTAL_UNITS                  float64
GROSS_SQUARE_FEET            float64
YEAR_BUILT                   float64
TAX_CLASS_AT_TIME_OF_SALE      int64
dtype: object

In [85]:
pd.isnull(y_train).sum() > 0

False

In [0]:
import numpy as np

X_train.replace(np.inf, np.nan)

In [82]:
X_train.fillna(method='ffill')

,BOROUGH_1,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
0,1,0,0,0,0,716,1246,10011.0,1.0,0.0,1.0,1979.0,2007.0,2
1,1,0,0,0,0,812,68,10018.0,0.0,6.0,6.0,15435.0,1920.0,4
2,1,0,0,0,0,839,69,10018.0,0.0,7.0,7.0,11332.0,1930.0,4
3,1,0,0,0,0,592,1041,10014.0,1.0,0.0,1.0,500.0,0.0,2
4,1,0,0,0,0,1379,1402,10065.0,1.0,0.0,1.0,6406.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18162,0,0,0,1,0,15763,13,11691.0,2.0,0.0,2.0,2268.0,1965.0,1
18163,0,0,0,1,0,5137,1243,11355.0,1.0,0.0,1.0,1151.0,0.0,2
18164,0,0,0,1,0,9854,3,11432.0,2.0,1.0,3.0,2800.0,1931.0,1
18165,0,0,0,1,0,8133,24,11363.0,1.0,0.0,1.0,1526.0,1925.0,1


In [86]:
# Feature Scaling

from sklearn.feature_selection import f_regression, SelectKBest

selector = SelectKBest(score_func=f_regression, k=10)

X_train_select = selector.fit_transform(X_train, y_train)
X_test_select = selector.transform(X_test)

ValueError: ignored